In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arogoai-assignment/professional_mental.csv
/kaggle/input/arogoai-assignment/student_mental.csv


In [2]:
!pip install pytorch_tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.1 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

In [ ]:
# TabNet code for professional

In [9]:
from pytorch_tabnet.tab_model import TabNetClassifier

data_path = "/kaggle/input/mental-health-arogoai/professional_mental.csv"
df = pd.read_csv(data_path)

if "Unnamed: 0.1" in df.columns:
    df.drop(["Unnamed: 0.1"], axis=1, inplace=True)
if "Unnamed: 0" in df.columns:
    df.drop(["Unnamed: 0"], axis=1, inplace=True)

target_col = "Depression"
X = df.drop(target_col, axis=1).values
y = df[target_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

clf = TabNetClassifier(
    input_dim=13,
    device_name=device.type,
    
    n_d=8,              
    n_a=8,              
    n_steps=5,          
    gamma=1.3,           
    lambda_sparse=1e-3, 
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.01),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax', 
)

max_epochs = 100
batch_size = 1024
virtual_batch_size = 128

clf.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=max_epochs,
    patience=15,  
    batch_size=batch_size,
    virtual_batch_size=virtual_batch_size,
    num_workers=0,
    drop_last=False
)

model_path = "network.zip"
clf.save_model(model_path)
print(f"Model saved to {model_path}")

preds = clf.predict(X_test)
pred_probs = clf.predict_proba(X_test)

acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds, average="weighted")  
print(f"Test Accuracy: {acc:.4f}")
print(f"Test F1 Score: {f1:.4f}")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.27158 | train_accuracy: 0.94484 | valid_accuracy: 0.94587 |  0:00:09s
epoch 1  | loss: 0.14491 | train_accuracy: 0.95032 | valid_accuracy: 0.95169 |  0:00:18s
epoch 2  | loss: 0.13575 | train_accuracy: 0.95214 | valid_accuracy: 0.95337 |  0:00:27s
epoch 3  | loss: 0.13113 | train_accuracy: 0.95248 | valid_accuracy: 0.95315 |  0:00:36s
epoch 4  | loss: 0.12915 | train_accuracy: 0.95378 | valid_accuracy: 0.95368 |  0:00:45s
epoch 5  | loss: 0.12547 | train_accuracy: 0.95435 | valid_accuracy: 0.95626 |  0:00:54s
epoch 6  | loss: 0.12353 | train_accuracy: 0.95547 | valid_accuracy: 0.95706 |  0:01:03s
epoch 7  | loss: 0.11986 | train_accuracy: 0.95599 | valid_accuracy: 0.95764 |  0:01:12s
epoch 8  | loss: 0.11849 | train_accuracy: 0.95686 | valid_accuracy: 0.95781 |  0:01:22s
epoch 9  | loss: 0.11948 | train_accuracy: 0.95684 | valid_accuracy: 0.95715 |  0:01:31s
epoch 10 | loss: 0.11624 | train_accuracy: 0.95655 | valid_accuracy: 0.95732 |  0:01:40s
epoch 11 | loss: 0.11

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at network.zip.zip
Model saved to network.zip
Test Accuracy: 0.9627
Test F1 Score: 0.9612


In [14]:
def hyperparameter_tuning(X_train, y_train, X_valid, y_valid):
    best_acc = 0
    best_params = {}
    for n_steps in [3, 5]:
        for lr in [1e-2, 2e-2, 5e-3]:
            print(f"Training with n_steps={n_steps}, lr={lr}")
            temp_clf = TabNetClassifier(
                device_name=device.type,
                n_d=8,
                n_a=8,
                n_steps=n_steps,
                gamma=1.3,
                lambda_sparse=1e-3,
                optimizer_fn=torch.optim.Adam,
                optimizer_params=dict(lr=lr),
                scheduler_params={"step_size":10, "gamma":0.9},
                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                mask_type='sparsemax',
            )
            temp_clf.fit(
                X_train, y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                eval_name=['train', 'valid'],
                eval_metric=['accuracy'],
                max_epochs=50,
                patience=10,
                batch_size=batch_size,
                virtual_batch_size=virtual_batch_size,
                num_workers=0,
                drop_last=False,
                
            )
            preds_valid = temp_clf.predict(X_valid)
            acc_valid = accuracy_score(y_valid, preds_valid)
            print(f"Validation Accuracy: {acc_valid:.4f}")
            if acc_valid > best_acc:
                best_acc = acc_valid
                best_params = {"n_steps": n_steps, "lr": lr}
    return best_params, best_acc

best_params, best_acc = hyperparameter_tuning(X_train, y_train, X_test, y_test)
print("Best Hyperparameters:", best_params)
print("Best Validation Accuracy:", best_acc)

Training with n_steps=3, lr=0.01


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.20151 | train_accuracy: 0.9528  | valid_accuracy: 0.95346 |  0:00:05s
epoch 1  | loss: 0.12985 | train_accuracy: 0.95428 | valid_accuracy: 0.95542 |  0:00:10s
epoch 2  | loss: 0.11879 | train_accuracy: 0.95795 | valid_accuracy: 0.9591  |  0:00:15s
epoch 3  | loss: 0.11364 | train_accuracy: 0.95879 | valid_accuracy: 0.95994 |  0:00:21s
epoch 4  | loss: 0.11169 | train_accuracy: 0.95956 | valid_accuracy: 0.96123 |  0:00:26s
epoch 5  | loss: 0.11073 | train_accuracy: 0.95958 | valid_accuracy: 0.96066 |  0:00:31s
epoch 6  | loss: 0.11034 | train_accuracy: 0.95982 | valid_accuracy: 0.95999 |  0:00:37s
epoch 7  | loss: 0.10964 | train_accuracy: 0.96021 | valid_accuracy: 0.96128 |  0:00:42s
epoch 8  | loss: 0.10917 | train_accuracy: 0.96018 | valid_accuracy: 0.96177 |  0:00:48s
epoch 9  | loss: 0.10892 | train_accuracy: 0.96042 | valid_accuracy: 0.96119 |  0:00:53s
epoch 10 | loss: 0.10893 | train_accuracy: 0.96001 | valid_accuracy: 0.96163 |  0:00:58s
epoch 11 | loss: 0.10

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Validation Accuracy: 0.9621
Training with n_steps=3, lr=0.02


...

Validation Accuracy: 0.9619
Training with n_steps=5, lr=0.005


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.33604 | train_accuracy: 0.9376  | valid_accuracy: 0.93752 |  0:00:07s
epoch 1  | loss: 0.16364 | train_accuracy: 0.94579 | valid_accuracy: 0.94507 |  0:00:15s
epoch 2  | loss: 0.14806 | train_accuracy: 0.94955 | valid_accuracy: 0.95066 |  0:00:23s
epoch 3  | loss: 0.13823 | train_accuracy: 0.95075 | valid_accuracy: 0.95164 |  0:00:30s
epoch 4  | loss: 0.13331 | train_accuracy: 0.95214 | valid_accuracy: 0.95213 |  0:00:38s
epoch 5  | loss: 0.12886 | train_accuracy: 0.95358 | valid_accuracy: 0.95404 |  0:00:45s
epoch 6  | loss: 0.12781 | train_accuracy: 0.95294 | valid_accuracy: 0.95302 |  0:00:53s
epoch 7  | loss: 0.12679 | train_accuracy: 0.95444 | valid_accuracy: 0.95599 |  0:01:00s
epoch 8  | loss: 0.12387 | train_accuracy: 0.95524 | valid_accuracy: 0.95599 |  0:01:08s
epoch 9  | loss: 0.12183 | train_accuracy: 0.9562  | valid_accuracy: 0.95746 |  0:01:16s
epoch 10 | loss: 0.12067 | train_accuracy: 0.95532 | valid_accuracy: 0.9551  |  0:01:23s
epoch 11 | loss: 0.12

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Validation Accuracy: 0.9615
Best Hyperparameters: {'n_steps': 5, 'lr': 0.01}
Best Validation Accuracy: 0.9624761312669301


In [7]:
from pytorch_tabnet.tab_model import TabNetClassifier

data_path = "/kaggle/input/arogoai-assignment/professional_mental.csv"
df = pd.read_csv(data_path)

if "Unnamed: 0.1" in df.columns:
    df.drop(["Unnamed: 0.1"], axis=1, inplace=True)
if "Unnamed: 0" in df.columns:
    df.drop(["Unnamed: 0"], axis=1, inplace=True)

target_col = "Depression"
X = df.drop(target_col, axis=1).values
y = df[target_col].values


scaler = StandardScaler()
X_train = scaler.fit_transform(X)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

clf = TabNetClassifier(
    input_dim=13,
    device_name=device.type,
    
    n_d=8,              
    n_a=8,              
    n_steps=5,          
    gamma=1.3,           
    lambda_sparse=1e-3, 
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.01),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax', 
)

max_epochs = 100
batch_size = 1024
virtual_batch_size = 128

clf.fit(
    X, y,
    max_epochs=max_epochs,
    patience=15,  
    batch_size=batch_size,
    virtual_batch_size=virtual_batch_size,
    num_workers=0,
    drop_last=False
)

model_path = "network.zip"
clf.save_model(model_path)
print(f"Model saved to {model_path}")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.24194 |  0:00:05s
epoch 1  | loss: 0.15228 |  0:00:11s
epoch 2  | loss: 0.14162 |  0:00:17s
epoch 3  | loss: 0.13267 |  0:00:22s
epoch 4  | loss: 0.12625 |  0:00:28s
epoch 5  | loss: 0.12454 |  0:00:33s
epoch 6  | loss: 0.12252 |  0:00:39s
epoch 7  | loss: 0.12105 |  0:00:45s
epoch 8  | loss: 0.12089 |  0:00:51s
epoch 9  | loss: 0.11925 |  0:00:56s
epoch 10 | loss: 0.11751 |  0:01:02s
epoch 11 | loss: 0.11568 |  0:01:07s
epoch 12 | loss: 0.11417 |  0:01:13s
epoch 13 | loss: 0.11179 |  0:01:19s
epoch 14 | loss: 0.11134 |  0:01:25s
epoch 15 | loss: 0.11008 |  0:01:30s
epoch 16 | loss: 0.10946 |  0:01:36s
epoch 17 | loss: 0.10875 |  0:01:41s
epoch 18 | loss: 0.10856 |  0:01:47s
epoch 19 | loss: 0.10801 |  0:01:53s
epoch 20 | loss: 0.10766 |  0:01:59s
epoch 21 | loss: 0.10759 |  0:02:04s
epoch 22 | loss: 0.10723 |  0:02:10s
epoch 23 | loss: 0.10797 |  0:02:16s
epoch 24 | loss: 0.10733 |  0:02:21s
epoch 25 | loss: 0.10803 |  0:02:27s
epoch 26 | loss: 0.10701 |  0:02:33s
e

In [ ]:
# TabNet on student dataset

In [4]:
from pytorch_tabnet.tab_model import TabNetClassifier

data_path = "/kaggle/input/mental-health-arogoai/student_mental.csv"
df = pd.read_csv(data_path)


if "Unnamed: 0.1" in df.columns:
    df.drop(["Unnamed: 0.1"], axis=1, inplace=True)
if "Unnamed: 0" in df.columns:
    df.drop(["Unnamed: 0"], axis=1, inplace=True)

target_col = "Depression"
X = df.drop(target_col, axis=1).values
y = df[target_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

clf = TabNetClassifier(
    device_name=device.type,
    n_d=8,              
    n_a=8,              
    n_steps=5,          
    gamma=1.3,           
    lambda_sparse=1e-3, 
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.01),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    mask_type='sparsemax', 
)

max_epochs = 100
batch_size = 1024
virtual_batch_size = 128

clf.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=max_epochs,
    patience=15,  
    batch_size=batch_size,
    virtual_batch_size=virtual_batch_size,
    num_workers=0,
    drop_last=False
)

model_path = "tabnet_model.zip"
clf.save_model(model_path)
print(f"Model saved to {model_path}")

preds = clf.predict(X_test)
pred_probs = clf.predict_proba(X_test)

acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds, average="weighted")  
print(f"Test Accuracy: {acc:.4f}")
print(f"Test F1 Score: {f1:.4f}")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.72072 | train_accuracy: 0.74182 | valid_accuracy: 0.75265 |  0:00:02s
epoch 1  | loss: 0.50392 | train_accuracy: 0.78866 | valid_accuracy: 0.80011 |  0:00:04s
epoch 2  | loss: 0.47328 | train_accuracy: 0.79382 | valid_accuracy: 0.79975 |  0:00:06s
epoch 3  | loss: 0.45391 | train_accuracy: 0.81117 | valid_accuracy: 0.81521 |  0:00:08s
epoch 4  | loss: 0.42666 | train_accuracy: 0.81643 | valid_accuracy: 0.82438 |  0:00:10s
epoch 5  | loss: 0.41529 | train_accuracy: 0.8203  | valid_accuracy: 0.82491 |  0:00:12s
epoch 6  | loss: 0.40502 | train_accuracy: 0.82605 | valid_accuracy: 0.83031 |  0:00:14s
epoch 7  | loss: 0.39603 | train_accuracy: 0.82583 | valid_accuracy: 0.83534 |  0:00:16s
epoch 8  | loss: 0.39156 | train_accuracy: 0.83005 | valid_accuracy: 0.83876 |  0:00:17s
epoch 9  | loss: 0.38867 | train_accuracy: 0.83634 | valid_accuracy: 0.83876 |  0:00:20s
epoch 10 | loss: 0.38105 | train_accuracy: 0.83724 | valid_accuracy: 0.84145 |  0:00:21s
epoch 11 | loss: 0.37

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_model.zip.zip
Model saved to tabnet_model.zip
Test Accuracy: 0.8508
Test F1 Score: 0.8501


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

df = pd.read_csv("/kaggle/input/mental-health-arogoai/student_mental.csv")
df.drop(columns=['Unnamed: 0'] , inplace=True)

In [13]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("xgboost").setLevel(logging.CRITICAL)

In [14]:
X = df.drop("Depression", axis=1)
y = df["Depression"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

space = {
    'max_depth': hp.quniform('max_depth', 3, 12, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

def objective(params):
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    model = XGBClassifier(
        booster='gbtree',
        objective='binary:logistic',
        use_label_encoder=False,
        eval_metric='logloss',
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        **params
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials, rstate=np.random.default_rng(42))
best['max_depth'] = int(best['max_depth'])
best['n_estimators'] = int(best['n_estimators'])
best['min_child_weight'] = int(best['min_child_weight'])

final_model = XGBClassifier(
    booster='gbtree',
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss',
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    **best
)
final_model.fit(X_train, y_train)
preds = final_model.predict(X_test)
print("Final Accuracy:", accuracy_score(y_test, preds))
print("Best Hyperparameters:", best)

100%|██████████| 50/50 [00:47<00:00,  1.04trial/s, best loss: -0.8565522200251663]
Final Accuracy: 0.8565522200251663
Best Hyperparameters: {'colsample_bytree': 0.5755485717971284, 'gamma': 0.02218720951511316, 'learning_rate': 0.16808878278453498, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 184, 'subsample': 0.5960466731264948}


In [23]:
import pickle
pickle.dump(final_model, open("xgboost_model.pkl", "wb"))

In [ ]:
# Random forest for student datasets

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [500, 700, 900],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1')
grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_

y_pred = best_rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

with open("random_forest_model.pkl", "wb") as file:
    pickle.dump(best_rf_model, file)

Accuracy: 84.24%
F1 Score: 0.87
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.77      0.80      2293
           1       0.85      0.89      0.87      3270

    accuracy                           0.84      5563
   macro avg       0.84      0.83      0.84      5563
weighted avg       0.84      0.84      0.84      5563



In [ ]:
# Tabtransformer for student dataset

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import TensorDataset, DataLoader
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

class TabTransformer(nn.Module):
    def __init__(self, num_features, embed_dim, depth, num_heads, dropout, mlp_hidden_dim, num_classes):
        super(TabTransformer, self).__init__()
        self.feature_projection = nn.Linear(1, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        self.mlp = nn.Sequential(
            nn.Linear(num_features * embed_dim, mlp_hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden_dim, num_classes)
        )
    def forward(self, x):
        x = x.unsqueeze(-1)
        x = self.feature_projection(x)
        x = self.transformer(x)
        x = x.flatten(1)
        return self.mlp(x)

def train_model(model, optimizer, criterion, loader, device):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    return total_loss / len(loader.dataset)

def evaluate_model(model, loader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            outputs = model(xb)
            preds.append(torch.argmax(outputs, dim=1).cpu().numpy())
            targets.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    return accuracy_score(targets, preds), f1_score(targets, preds, average='weighted')

In [21]:

df = pd.read_csv("/kaggle/input/mental-health-arogoai/student_mental.csv")
X = df.drop("Depression", axis=1).values
y = df["Depression"].values

In [22]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
batch_size = 32
train_loader = DataLoader(TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long)), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)), batch_size=batch_size, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def objective(params):
    embed_dim = int(params['embed_dim'])
    depth = int(params['depth'])
    num_heads = int(params['num_heads'])
    dropout = params['dropout']
    mlp_hidden_dim = int(params['mlp_hidden_dim'])
    lr = params['lr']
    if embed_dim % num_heads != 0:
        return {'loss': 1.0, 'status': STATUS_OK}
    model = TabTransformer(num_features=X_train.shape[1], embed_dim=embed_dim, depth=depth, num_heads=num_heads, dropout=dropout, mlp_hidden_dim=mlp_hidden_dim, num_classes=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    epochs = 10
    for _ in range(epochs):
        train_model(model, optimizer, criterion, train_loader, device)
    acc, _ = evaluate_model(model, val_loader, device)
    return {'loss': -acc, 'status': STATUS_OK}

space = {
    'embed_dim': hp.quniform('embed_dim', 16, 64, 8),
    'depth': hp.quniform('depth', 1, 4, 1),
    'num_heads': hp.quniform('num_heads', 1, 4, 1),
    'dropout': hp.uniform('dropout', 0.1, 0.5),
    'mlp_hidden_dim': hp.quniform('mlp_hidden_dim', 32, 256, 32),
    'lr': hp.loguniform('lr', np.log(1e-4), np.log(1e-2))
}

trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20, trials=trials, rstate=np.random.default_rng(42))
best['embed_dim'] = int(best['embed_dim'])
best['depth'] = int(best['depth'])
best['num_heads'] = int(best['num_heads'])
best['mlp_hidden_dim'] = int(best['mlp_hidden_dim'])

model = TabTransformer(num_features=X_train.shape[1], embed_dim=best['embed_dim'], depth=best['depth'], num_heads=best['num_heads'], dropout=best['dropout'], mlp_hidden_dim=best['mlp_hidden_dim'], num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=best['lr'])
criterion = nn.CrossEntropyLoss()
epochs = 25
for _ in range(epochs):
    train_model(model, optimizer, criterion, train_loader, device)
val_acc, val_f1 = evaluate_model(model, val_loader, device)
test_acc, test_f1 = evaluate_model(model, test_loader, device)
print("Validation Accuracy:", val_acc, "Validation F1:", val_f1)
print("Test Accuracy:", test_acc, "Test F1:", test_f1)

100%|██████████| 20/20 [10:37<00:00, 31.86s/trial, best loss: -0.8542665388302972]
Validation Accuracy: 0.8511505273250239 Validation F1: 0.8506410884160949
Test Accuracy: 0.8487535953978907 Test F1: 0.8482669857844034
